<a href="https://colab.research.google.com/github/sakesrinivas/telemedicine-kiosk/blob/main/Copy_of_hf_model_evaluation_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hugging Face Model Evaluation in Google Colab

In [ ]:
!pip -q install transformers datasets rouge-score bert-score nltk pandas torch

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.6 MB/s eta 0:00:00
   

In [ ]:
import json
import nltk
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from nltk.tokenize import sent_tokenize
from rouge_score import rouge_scorer
from bert_score import score
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes
!pip install --upgrade accelerate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 115.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.3.0
    Uninstalling accelerate-1.3.0:
      Successfully uninstalled accelerate-1.3.0


In [ ]:
import json
import nltk
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from nltk.tokenize import sent_tokenize
from rouge_score import rouge_scorer
from bert_score import score
import bitsandbytes
# Download NLTK tokenizer
nltk.download('punkt')

# Load the dataset from Hugging Face
def load_dataset_from_hf(dataset_name):
    dataset = load_dataset(dataset_name, split="train")

    conversations = []
    for convo in dataset["conversations"]:
        human_turns = []
        assistant_turns = []
        for turn in convo:
            if turn["from"] == "human":
                human_turns.append(turn["value"])
            else:
                assistant_turns.append(turn["value"])

        # Combine full conversation into a single input-response pair
        if human_turns and assistant_turns:
            conversations.append((" ".join(human_turns), " ".join(assistant_turns)))

    print(f"Loaded {len(conversations)} multi-turn conversations.")
    return conversations

# Load Hugging Face model and tokenizer
def load_hf_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")
    return model, tokenizer

# Query Hugging Face model
def query_hf_model(prompt, model, tokenizer):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(input_ids, max_length=512, do_sample=True, temperature=0.5)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

# Evaluate Conv-ROUGE
def evaluate_conv_rouge(reference, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return {
        "Conv-ROUGE-1": scores["rouge1"].fmeasure,
        "Conv-ROUGE-2": scores["rouge2"].fmeasure,
        "Conv-ROUGE-L": scores["rougeL"].fmeasure
    }

# Evaluate BERTScore
def evaluate_bertscore(reference, generated):
    P, R, F1 = score([generated], [reference], lang="en", rescale_with_baseline=True)
    return F1.item()  # Return F1-score

# Run Evaluation
def evaluate_model(dataset_name, model_name, output_file):
    model, tokenizer = load_hf_model(model_name)
    conversation_pairs = load_dataset_from_hf(dataset_name)
    results = []

    import random
    sampled_conversations = random.sample(conversation_pairs, min(5, len(conversation_pairs)))
    for user_inputs, expected_response in sampled_conversations:  # Limit to 100 samples
        generated_response = query_hf_model(user_inputs, model, tokenizer)

        # Compute Conv-ROUGE
        conv_rouge_scores = evaluate_conv_rouge(expected_response, generated_response)

        # Compute BERTScore
        bert_score = evaluate_bertscore(expected_response, generated_response)

        # Store results
        results.append({
            "User Input": user_inputs,
            "Expected Response": expected_response,
            "Generated Response": generated_response,
            "Conv-ROUGE-1": conv_rouge_scores["Conv-ROUGE-1"],
            "Conv-ROUGE-2": conv_rouge_scores["Conv-ROUGE-2"],
            "Conv-ROUGE-L": conv_rouge_scores["Conv-ROUGE-L"],
            "BERTScore": bert_score
        })

    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)
    print(f"Evaluation results saved to {output_file}")

# Main Execution
if __name__ == "__main__":
    dataset_name = "savinirsekas/MediTalk-300-Conversational"  # Replace with your actual Hugging Face dataset
    model_name = "savinirsekas/MediTalk-300-Mistral-7B-4bit"  # Replace with your actual Hugging Face model
    output_file = "conv_rouge_bertscore_results_hf_mistral7b-4bit_ft.csv"

    evaluate_model(dataset_name, model_name, output_file)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

MediTalk-300-Conversational.json:   0%|          | 0.00/4.18M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1752 [00:00<?, ? examples/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Evaluation results saved to conv_rouge_bertscore_results_hf_mistral7b-4bit_ft.csv


In [ ]:
import json
import nltk
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from nltk.tokenize import sent_tokenize
from rouge_score import rouge_scorer
from bert_score import score

# Download NLTK tokenizer
nltk.download('punkt')

# Load the dataset from Hugging Face
def load_dataset_from_hf(dataset_name):
    dataset = load_dataset(dataset_name, split="train")

    conversations = []
    for convo in dataset["conversations"]:
        human_turns = []
        assistant_turns = []
        for turn in convo:
            if turn["from"] == "human":
                human_turns.append(turn["value"])
            else:
                assistant_turns.append(turn["value"])

        # Combine full conversation into a single input-response pair
        if human_turns and assistant_turns:
            conversations.append((" ".join(human_turns), " ".join(assistant_turns)))

    print(f"Loaded {len(conversations)} multi-turn conversations.")
    return conversations

# Load Hugging Face model and tokenizer
def load_hf_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        offload_folder="offload_weights"  # Added offload folder
    )
    return model, tokenizer

# Query Hugging Face model
def query_hf_model(prompt, model, tokenizer):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)
    output_ids = model.generate(input_ids, max_length=512, do_sample=True, temperature=0.5)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

# Evaluate Conv-ROUGE
def evaluate_conv_rouge(reference, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return {
        "Conv-ROUGE-1": scores["rouge1"].fmeasure,
        "Conv-ROUGE-2": scores["rouge2"].fmeasure,
        "Conv-ROUGE-L": scores["rougeL"].fmeasure
    }

# Evaluate BERTScore
def evaluate_bertscore(reference, generated):
    P, R, F1 = score([generated], [reference], lang="en", rescale_with_baseline=True)
    return F1.item()  # Return F1-score

# Run Evaluation
def evaluate_model(dataset_name, model_name, output_file):
    model, tokenizer = load_hf_model(model_name)
    conversation_pairs = load_dataset_from_hf(dataset_name)
    results = []

    import random
    sampled_conversations = random.sample(conversation_pairs, min(5, len(conversation_pairs)))
    for user_inputs, expected_response in sampled_conversations:  # Limit to 100 samples
        generated_response = query_hf_model(user_inputs, model, tokenizer)

        # Compute Conv-ROUGE
        conv_rouge_scores = evaluate_conv_rouge(expected_response, generated_response)

        # Compute BERTScore
        bert_score = evaluate_bertscore(expected_response, generated_response)

        # Store results
        results.append({
            "User Input": user_inputs,
            "Expected Response": expected_response,
            "Generated Response": generated_response,
            "Conv-ROUGE-1": conv_rouge_scores["Conv-ROUGE-1"],
            "Conv-ROUGE-2": conv_rouge_scores["Conv-ROUGE-2"],
            "Conv-ROUGE-L": conv_rouge_scores["Conv-ROUGE-L"],
            "BERTScore": bert_score
        })

    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)
    print(f"Evaluation results saved to {output_file}")

# Main Execution
if __name__ == "__main__":
    dataset_name = "savinirsekas/MediTalk-300-Conversational"  # Replace with your actual Hugging Face dataset
    model_name = "savinirsekas/MediTalk-300-Gemma2-9B"  # Replace with your actual Hugging Face model
    output_file = "conv_rouge_bertscore_results_hf_gemma9B_ft.csv"

    evaluate_model(dataset_name, model_name, output_file)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded 1752 multi-turn conversations.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

Evaluation results saved to conv_rouge_bertscore_results_hf_gemma9B_ft.csv


# **Summarizing/Evaluating Full Dataset**

In [ ]:
import json
import nltk
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from nltk.tokenize import sent_tokenize
from rouge_score import rouge_scorer
from bert_score import score
import requests

# Download NLTK tokenizer
nltk.download('punkt')

# Load the fine-tuned model and tokenizer with CPU offloading
def load_hf_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        offload_folder="offload_weights"
    )
    return model, tokenizer

# Generate summaries using Hugging Face model with optimizations
def generate_summaries_with_hf(prompts, model, tokenizer, batch_size=1):
    summaries = []
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i + batch_size]
        with torch.no_grad():  # Disable gradient computation
            input_ids = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)
            output_ids = model.generate(
                input_ids, max_length=512, do_sample=True, top_p=0.7, temperature=0.6, repetition_penalty=1.2, no_repeat_ngram_size=3
            )
            summaries.extend(tokenizer.batch_decode(output_ids, skip_special_tokens=True))
        torch.cuda.empty_cache()  # Free memory after processing each batch
    return summaries

# Summarize conversation into user and assistant perspectives with smaller batch size
def summarize_conversations(conversations, model, tokenizer):
    user_prompts = ["Summarize as the user:\n" + "\n".join([turn["value"] for turn in convo if turn["from"] == "human"]) for convo in conversations]
    assistant_prompts = ["Summarize as the assistant:\n" + "\n".join([turn["value"] for turn in convo if turn["from"] == "assistant"]) for convo in conversations]

    user_summaries = generate_summaries_with_hf(user_prompts, model, tokenizer, batch_size=1)
    assistant_summaries = generate_summaries_with_hf(assistant_prompts, model, tokenizer, batch_size=1)

    return list(zip(user_summaries, assistant_summaries))

# Evaluate Conv-ROUGE
def evaluate_conv_rouge(references, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return [{
        "Conv-ROUGE-1": scores["rouge1"].fmeasure,
        "Conv-ROUGE-2": scores["rouge2"].fmeasure,
        "Conv-ROUGE-L": scores["rougeL"].fmeasure
    } for scores in map(lambda x: scorer.score(x[0], x[1]), zip(references, generated))]

# Evaluate BERTScore with batching
def evaluate_bertscore(references, generated):
    with torch.no_grad():
        _, _, F1 = score(generated, references, lang="en", batch_size=min(2, len(generated)), rescale_with_baseline=True)
    torch.cuda.empty_cache()  # Free memory after BERTScore computation
    return F1.tolist()

# Load dataset and process conversations
def load_dataset_from_hf(dataset_name, model, tokenizer):
    dataset = load_dataset(dataset_name, split="train")
    conversations = [convo for convo in dataset["conversations"] if isinstance(convo, list) and len(convo) >= 2]
    summarized_conversations = summarize_conversations(conversations, model, tokenizer)
    print(f"Loaded {len(summarized_conversations)} summarized user-assistant pairs.")
    return summarized_conversations

# Run Evaluation
def evaluate_model(dataset_name, model_name, output_file):
    model, tokenizer = load_hf_model(model_name)
    conversation_pairs = load_dataset_from_hf(dataset_name, model, tokenizer)
    results = []

    import random
    sampled_conversations = random.sample(conversation_pairs, min(5, len(conversation_pairs)))
    user_summaries, expected_responses = zip(*sampled_conversations)

    # Generate responses one-by-one to reduce memory usage
    generated_responses = generate_summaries_with_hf(["Reply concisely:\n" + user for user in user_summaries], model, tokenizer, batch_size=1)

    # Compute Conv-ROUGE in batch
    conv_rouge_scores = evaluate_conv_rouge(expected_responses, generated_responses)

    # Compute BERTScore in batch
    bert_scores = evaluate_bertscore(expected_responses, generated_responses)

    # Store results
    for i in range(len(user_summaries)):
        results.append({
            "User Summary": user_summaries[i],
            "Expected Response": expected_responses[i],
            "Generated Response": generated_responses[i],
            "Conv-ROUGE-1": conv_rouge_scores[i]["Conv-ROUGE-1"],
            "Conv-ROUGE-2": conv_rouge_scores[i]["Conv-ROUGE-2"],
            "Conv-ROUGE-L": conv_rouge_scores[i]["Conv-ROUGE-L"],
            "BERTScore": bert_scores[i]
        })

    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)
    print(f"Evaluation results saved to {output_file}")

# Main Execution
if __name__ == "__main__":
    dataset_name = "savinirsekas/MediTalk-300-Conversational"  # Replace with your dataset
    model_name = "savinirsekas/MediTalk-300-Mistral-7B-4bit"  # Replace with your model
    output_file = "conv_rouge_bertscore_results_hf_mistral_ft_summ.csv"

    evaluate_model(dataset_name, model_name, output_file)


# **Summarizing and Evaluating Select Number of Conversations**

In [ ]:
import json
import nltk
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from nltk.tokenize import sent_tokenize
from rouge_score import rouge_scorer
from bert_score import score
import requests
import random

# Download NLTK tokenizer
nltk.download('punkt')

# Load the fine-tuned model and tokenizer with CPU offloading
def load_hf_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        offload_folder="offload_weights"
    )
    return model, tokenizer

# Generate summaries using Hugging Face model with optimizations
def generate_summaries_with_hf(prompts, model, tokenizer, batch_size=1):
    summaries = []
    for i in range(0, len(prompts), batch_size):
        batch = prompts[i:i + batch_size]
        with torch.no_grad():  # Disable gradient computation
            input_ids = tokenizer(batch, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)
            output_ids = model.generate(
                input_ids, max_new_tokens=525, do_sample=True, top_p=0.7, temperature=0.6, repetition_penalty=1.2, no_repeat_ngram_size=3
            )
            summaries.extend(tokenizer.batch_decode(output_ids, skip_special_tokens=True))
        torch.cuda.empty_cache()  # Free memory after processing each batch
    return summaries

# Summarize conversation into user and assistant perspectives with smaller batch size
def summarize_conversations(conversations, model, tokenizer):
    user_prompts = ["Summarize as the user:\n" + "\n".join([turn["value"] for turn in convo if turn["from"] == "human"]) for convo in conversations]
    assistant_prompts = ["Summarize as the assistant:\n" + "\n".join([turn["value"] for turn in convo if turn["from"] == "assistant"]) for convo in conversations]

    user_summaries = generate_summaries_with_hf(user_prompts, model, tokenizer, batch_size=1)
    assistant_summaries = generate_summaries_with_hf(assistant_prompts, model, tokenizer, batch_size=1)

    return list(zip(user_summaries, assistant_summaries))

# Evaluate Conv-ROUGE
def evaluate_conv_rouge(references, generated):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    return [{
        "Conv-ROUGE-1": scores["rouge1"].fmeasure,
        "Conv-ROUGE-2": scores["rouge2"].fmeasure,
        "Conv-ROUGE-L": scores["rougeL"].fmeasure
    } for scores in map(lambda x: scorer.score(x[0], x[1]), zip(references, generated))]

# Evaluate BERTScore with batching
"""def evaluate_bertscore(references, generated):
    with torch.no_grad():
        _, _, F1 = score(generated, references, lang="en", batch_size=min(2, len(generated)), rescale_with_baseline=True)
    torch.cuda.empty_cache()  # Free memory after BERTScore computation
    return F1.tolist()"""


def evaluate_bertscore(references, generated):
    with torch.no_grad():
        _, _, F1 = score(tuple(generated), tuple(references), lang="en", batch_size=min(2, len(generated)), rescale_with_baseline=True)
    torch.cuda.empty_cache()  # Free memory after BERTScore computation
    return F1.tolist()



# Load dataset and process conversations
def load_dataset_from_hf(dataset_name, model, tokenizer):
    dataset = load_dataset(dataset_name, split="train")
    conversations = [convo for convo in dataset["conversations"] if isinstance(convo, list) and len(convo) >= 2]
    #conversations = conversations[:5]  # Limit to Load First 5 conversations for summarization
    conversations = random.sample(conversations, min(25, len(conversations)))
    summarized_conversations = summarize_conversations(conversations, model, tokenizer)
    print(f"Loaded {len(summarized_conversations)} summarized user-assistant pairs.")
    return summarized_conversations

# Run Evaluation
def evaluate_model(dataset_name, model_name, output_file):
    model, tokenizer = load_hf_model(model_name)
    conversation_pairs = load_dataset_from_hf(dataset_name, model, tokenizer)
    results = []

    # Only evaluate the 5 summarized conversations
    user_summaries, expected_responses = zip(*conversation_pairs)

    # Generate responses one-by-one to reduce memory usage
    generated_responses = generate_summaries_with_hf(["Reply Concisely:\n" + user for user in user_summaries], model, tokenizer, batch_size=1)

    # Compute Conv-ROUGE in batch
    conv_rouge_scores = evaluate_conv_rouge(expected_responses, generated_responses)

    # Compute BERTScore in batch
    bert_scores = evaluate_bertscore(expected_responses, generated_responses)

    # Store results
    for i in range(len(user_summaries)):
        results.append({
            "User Summary": user_summaries[i],
            "Expected Response": expected_responses[i],
            "Generated Response": generated_responses[i],
            "Conv-ROUGE-1": conv_rouge_scores[i]["Conv-ROUGE-1"],
            "Conv-ROUGE-2": conv_rouge_scores[i]["Conv-ROUGE-2"],
            "Conv-ROUGE-L": conv_rouge_scores[i]["Conv-ROUGE-L"],
            "BERTScore": bert_scores[i]
        })

    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False)
    print(f"Evaluation results saved to {output_file}")

# Main Execution
if __name__ == "__main__":
    dataset_name = "savinirsekas/MediTalk-300-Conversational"  # Replace with your dataset
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit"  # Replace with your model
    output_file = "conv_rouge_bertscore_results_llama3-8b-it-bnb-4bit_25.csv"

    evaluate_model(dataset_name, model_name, output_file)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Loaded 25 summarized user-assistant pairs.


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Evaluation results saved to conv_rouge_bertscore_results_gemma-7b-it-bnb-4bit_summ25.csv
